This notebook is used for testing the posibilities of the data augmentation in keras. The functionality will added to the 2D/3D networks once the tests are done. 

In [ ]:
# general imports
import sys,os,time,random
from os import walk
import pickle
from __future__ import print_function

# scientific computing
import numpy as np
import matplotlib
matplotlib.use('Agg');
import matplotlib.pyplot as plt
plt.set_cmap('jet');
import skimage
from skimage.io import imread, imsave
import scipy

# machine learning stuff
import tensorflow as tf
import keras 
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
%matplotlib inline

In [ ]:
print("keras", keras.__version__)
print("tensorflow", tf.__version__)

In [ ]:
# check the backend the ordering of the channels
print(keras.backend.backend())
print(keras.backend.image_dim_ordering())
print(K.image_data_format())

In [ ]:
# this part is necessary to set the params from the command line
import sys
import argparse

if len(sys.argv) == 3 and sys.argv[1] == "-f": #on jupyter
    sys.argv = ['']
    
parser = argparse.ArgumentParser(description='Count-ception')

parser.add_argument('-seed', type=int, nargs='?',default=0, help='random seed for split and init')
parser.add_argument('-data', type=str, nargs='?',default="cells", help='Dataset folder')
parser.add_argument('-framesize', type=int, nargs='?',default=256, help='Size of the images processed at once')
parser.add_argument('-batchsize', type=int, nargs='?',default=32, help='Size of the batch in the generator')
parser.add_argument('-numbatches', type=int, nargs='?',default=5, help='Num of batches in the generator')

args = parser.parse_args()
print(args)

In [ ]:
# how much to extend the initial image 
patch_size = int(32)
framesize = int(args.framesize)
framesize_h = framesize_w = framesize
channels = int(3)
batch_size = int(args.batchsize)
num_batches = int(args.numbatches)

In [ ]:
# reproducibility
seed = args.seed
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
# piclked names
paramfilename = str(patch_size) + "-" + str(framesize) + "-" + str(batch_size) + "-" + str(num_batches) + "-" + args.data + "-generator-params.p"
datasetfilename = str(patch_size) + "-" + str(framesize) + "-" + str(batch_size) + "-" + str(num_batches) + "-" + args.data + "-generator-dataset.p"
print(paramfilename)
print(datasetfilename)

In [ ]:
# input shape is the image shape without the patches
input_shape = (framesize, framesize, channels)
ext_shape = (input_shape[0] + patch_size, input_shape[0] + patch_size, 1)
print (input_shape, ext_shape)

In [ ]:
# read the files from the test folder 
import glob

# prefix = '/Users/kkolyva/'
prefix = '/home/milkyklim/'
folder = prefix + 'dl-cell-counting/algorithm/data/test-cells'
folder_images = "" # 'images/'
folder_labels = "" # 'labels/'
img_ext = '.png'

print('Images path:', folder + folder_images)
print('Labels path:', folder + folder_labels)

imgs = []

for filename in glob.iglob(folder + folder_labels + "/*dots" + img_ext):
    imgg = filename.replace(folder_labels + "dots", folder_images + "cell")
    imgs.append([imgg,filename])
    
if len(imgs) == 0:
    print("Issue with dataset")
    sys.exit()

In [ ]:
for path in imgs: 
    if (not os.path.isfile(path[0])):
        print(path, "bad", path[0])
    if (not os.path.isfile(path[1])):
        print(path, "bad", path[1])

In [ ]:
dataset = [] 
if (os.path.isfile(datasetfilename)):
    print("reading", datasetfilename)
    dataset = pickle.load(open(datasetfilename, "rb" ))
else:
    dataset_x = []
    dataset_l = []
    print(len(imgs))
    for path in imgs:
        imgPath = path[0]
        print(imgPath)
        img = imread(imgPath)
        labelPath = path[1]
        markers = imread(labelPath)
        
        print("img shape", img.shape, "markers shape", markers.shape)
        
        if img.shape[0:2] != (framesize_w,framesize_h):
            print("!!!! Not adding image because size is" , img.shape[0:3])
        else:
            dataset.append((img, markers))        
            print("img shape", img.shape, "markers shape", markers.shape)
        
        sys.stdout.flush()
    
    print("writing", datasetfilename)
    out = open(datasetfilename, "wb",0)
    pickle.dump(dataset, out)
    print("dataset size", len(dataset))        
print("DONE")

In [ ]:
# grab the data from the data set
np.random.shuffle(dataset)

np_dataset_x = np.asarray([d[0] for d in dataset], dtype=np.float32)
np_dataset_l = np.asarray([d[1] for d in dataset], dtype=np.float32)

# np_dataset_x = np_dataset_x.transpose((0,3,1,2))

print("np_dataset_x", np_dataset_x.shape)
print("np_dataset_l", np_dataset_l.shape)

In [ ]:
length = len(np_dataset_x)
n = length

np_dataset_x_train = np_dataset_x[:n]
np_dataset_l_train = np_dataset_l[:n]

print("total number of initial images: ", len(np_dataset_x_train))

In [ ]:
# plot 2 images to check that the labels are correct 
# DEBUG:
plt.Figure(figsize=(5, 3), dpi=300)
plt.subplot(121)
img1 = np.array(np_dataset_x_train[1, ].reshape(input_shape), dtype=np.float32).astype(np.uint8)
plt.imshow(img1, cmap=plt.get_cmap('gray'))
plt.subplot(122)
img3 = np.array(np_dataset_l_train[1, ].reshape(input_shape), dtype=np.float32).astype(np.uint8)
plt.imshow(img3, cmap=plt.get_cmap('gray'))
print(np.sum(img3) / 255)

In [ ]:
# in case of images and labels we have to use the following approach: 
# create 2 instances with the same arguments: 
shift = 0.02
image_data_gen_args = dict(featurewise_center=False, 
                             featurewise_std_normalization=False, 
                             # zca_whitening=True, 
                             rotation_range=5,
                             width_shift_range=shift, 
                             height_shift_range=shift,
                             fill_mode = 'wrap',
                             # cval = 0,
                             horizontal_flip=True, 
                             vertical_flip=True)

datagen = ImageDataGenerator(**image_data_gen_args)
labelgen = ImageDataGenerator(**image_data_gen_args)

datagen.fit(np_dataset_x_train, augment=True, seed=seed)
labelgen.fit(np_dataset_l_train, augment=True, seed=seed)

image_generator = datagen.flow(
    np_dataset_x_train,
    seed=seed, 
    batch_size = batch_size,
    save_to_dir='data/aug', save_prefix='cell', save_format='png')
label_generator = labelgen.flow(
    np_dataset_l_train,
    seed=seed,
    batch_size = batch_size,
    save_to_dir='data/aug', save_prefix='dots', save_format='png')

j_batch = 0
for X_batch, y_batch in zip(image_generator, label_generator):
    print(j_batch, "th batch with the shapes:", X_batch.shape, y_batch.shape)     
    j_batch += 1;
    if (j_batch >= num_batches):
        break